# Implementing Convolutional Neural Network on OCR Dataset

In [71]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras import backend as K
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import math

import glob
import os
import cv2
import sys

In [73]:
data = pd.read_csv("letter-recognition.csv",header=0)

labelencoder = LabelEncoder()
data['letter'] = labelencoder.fit_transform(data['letter']) 

data.head(10)

,letter,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgey,yedge,yedgex
0,19,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,8,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,3,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,13,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,6,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10
5,18,4,11,5,8,3,8,8,6,9,5,6,6,0,8,9,7
6,1,4,2,5,4,4,8,7,6,6,7,6,6,2,8,7,10
7,0,1,1,3,2,1,8,2,2,2,8,2,8,1,6,2,7
8,9,2,2,4,4,2,10,6,2,6,12,4,8,1,6,1,7
9,12,11,15,13,9,7,13,2,6,2,12,1,9,8,1,1,8


In [74]:
# We share train data because this "sample_submission.csv" only has one class/label
x = data.iloc[:, 1:]
y = data['letter'].tolist()

# # Select 10000 rows data as a testing dataset
x_test = x.iloc[0:10000, :].values.astype('float32') # all pixel values 
y_test = y[0:10000] # Select label for testing data
x_train = x.iloc[10000:, :].values.astype('float32') # all pixel values 
y_train = y[10000:]

In [75]:
y_train=np.array(y_train)
y_test=np.array(y_test)

In [76]:
print('Train data: X=%s, y=%s' % (x_train.shape, y_train.shape))
print('Test data: X=%s, y=%s' % (x_test.shape, y_test.shape))

Train data: X=(10000, 16), y=(10000,)
Test data: X=(10000, 16), y=(10000,)


In [77]:
x_train = x_train.reshape((x_train.shape[0],x_train.shape[1], 1))
x_test  = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

In [78]:
print('Train data: X=%s, y=%s' % (x_train.shape, y_train.shape))
print('Test data: X=%s, y=%s' % (x_test.shape, y_test.shape))

Train data: X=(10000, 16, 1), y=(10000,)
Test data: X=(10000, 16, 1), y=(10000,)


In [79]:
#CHECK SHAPE DATA
print('x train shape : ', x_train.shape)
print('y_train shape : ', y_train.shape)
print('x_test shape : ', x_test.shape)
print('y_test shape : ', y_test.shape)

x train shape :  (10000, 16, 1)
y_train shape :  (10000,)
x_test shape :  (10000, 16, 1)
y_test shape :  (10000,)


In [80]:
y_train

array([22,  9,  3, ..., 19, 18,  0])

In [81]:
x_train

array([[[ 6.],
        [ 9.],
        [ 9.],
        ...,
        [11.],
        [ 0.],
        [ 8.]],

       [[ 2.],
        [ 9.],
        [ 3.],
        ...,
        [ 6.],
        [ 0.],
        [ 8.]],

       [[ 5.],
        [10.],
        [ 5.],
        ...,
        [10.],
        [ 6.],
        [ 6.]],

       ...,

       [[ 6.],
        [ 9.],
        [ 6.],
        ...,
        [12.],
        [ 2.],
        [ 4.]],

       [[ 2.],
        [ 3.],
        [ 4.],
        ...,
        [ 9.],
        [ 5.],
        [ 8.]],

       [[ 4.],
        [ 9.],
        [ 6.],
        ...,
        [ 7.],
        [ 2.],
        [ 8.]]], dtype=float32)

In [83]:
#normalize inputs:
x_train2 = x_train/255 - 0.5
x_test2 = x_test/255 - 0.5

# Convert class labels to one-hot encoded
y_train2 = keras.utils.to_categorical(y_train)
y_test2 = keras.utils.to_categorical(y_test)

letter_classes = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25]

y_train_indexes = []
for classs in y_train:
    y_train_indexes.append(letter_classes.index(classs))
    
# convert class labels to one-hot encoded:
y_train2 = keras.utils.to_categorical(y_train_indexes)

In [88]:
#import necessary building blocks:
from statistics import mean
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization
from keras.layers import LeakyReLU
from tensorflow.keras import activations

In [90]:
# model
model=Sequential()
# layers
model.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(16,1)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv1D(filters=64, kernel_size=2, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='softmax'))

model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 15, 32)            96        
                                                                 
 batch_normalization (BatchN  (None, 15, 32)           128       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 15, 32)            0         
                                                                 
 conv1d_1 (Conv1D)           (None, 14, 64)            4160      
                                                                 
 batch_normalization_1 (Batc  (None, 14, 64)           256       
 hNormalization)                                                 
                                                                 
 dropout_1 (Dropout)         (None, 14, 64)          

In [91]:
# compiling the model
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=0.005),loss='categorical_crossentropy',metrics=['accuracy'])

In [92]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2295575507032470054
xla_global_id: -1
]


In [93]:
#fitting the model
history=model.fit(x_train,y_train,epochs=10, validation_data=(x_test,y_test))


Epoch 1/10
313/313 [==============================] - 8s 16ms/step - loss: 0.0000e+00 - accuracy: 0.0372 - val_loss: 0.0000e+00 - val_accuracy: 0.0394
Epoch 2/10
313/313 [==============================] - 4s 14ms/step - loss: 0.0000e+00 - accuracy: 0.0372 - val_loss: 0.0000e+00 - val_accuracy: 0.0394
Epoch 3/10
313/313 [==============================] - 4s 14ms/step - loss: 0.0000e+00 - accuracy: 0.0372 - val_loss: 0.0000e+00 - val_accuracy: 0.0394
Epoch 4/10
313/313 [==============================] - 4s 14ms/step - loss: 0.0000e+00 - accuracy: 0.0372 - val_loss: 0.0000e+00 - val_accuracy: 0.0394
Epoch 5/10
313/313 [==============================] - 4s 14ms/step - loss: 0.0000e+00 - accuracy: 0.0372 - val_loss: 0.0000e+00 - val_accuracy: 0.0394
Epoch 6/10
313/313 [==============================] - 4s 14ms/step - loss: 0.0000e+00 - accuracy: 0.0372 - val_loss: 0.0000e+00 - val_accuracy: 0.0394
Epoch 7/10
313/313 [==============================] - 4s 14ms/step - loss: 0.0000e+00 - accura

In [94]:
# Predict using testing data without labels/classes
y_pred_test = model.predict(x_test2)
y_pred_test_classes = np.argmax(y_pred_test, axis=1) # Change to normal classes
y_pred_test_classes

313/313 [==============================] - 4s 3ms/step


array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [95]:
# Create the same format for actual classes
y_actual_test_classes = np.argmax(y_test2, axis=1) # Change to normal classes
y_actual_test_classes

array([19,  8,  3, ...,  9, 22, 16], dtype=int64)

In [96]:
from sklearn.metrics import multilabel_confusion_matrix
from math import sqrt

# Actual and predicted classes
lst_actual_class = y_actual_test_classes
lst_predicted_class = y_pred_test_classes

# Class = Label 0 to 9
lst_classes = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25]

# Compute multi-class confusion matrix
arr_out_matrix = multilabel_confusion_matrix(lst_actual_class, lst_predicted_class, labels=lst_classes)

# Temp store results
store_sens = [];
store_spec = [];
store_acc = [];
store_bal_acc = [];
store_prec = [];
store_fscore = [];
store_mcc = [];
for no_class in range(len(lst_classes)):
    arr_data = arr_out_matrix[no_class];
    print("Print Class: {0}".format(no_class));

    tp = arr_data[1][1]
    fp = arr_data[0][1]
    tn = arr_data[0][0]
    fn = arr_data[1][0]
    
    sensitivity = round(tp/(tp+fn), 3);
    specificity = round(tn/(tn+fp), 3);
    accuracy = round((tp+tn)/(tp+fp+tn+fn), 3);
    balanced_accuracy = round((sensitivity+specificity)/2, 3);
    precision = round(tp/(tp+fp), 3);
    f1Score = round((2*tp/(2*tp + fp + fn)), 3);
    x = (tp+fp) * (tp+fn) * (tn+fp) * (tn+fn)
    MCC = round(((tp * tn) - (fp * fn)) / sqrt(x), 3)
    store_sens.append(sensitivity);
    store_spec.append(specificity);
    store_acc.append(accuracy);
    store_bal_acc.append(balanced_accuracy);
    store_prec.append(precision);
    store_fscore.append(f1Score);
    store_mcc.append(MCC);
    print("TP={0}, FP={1}, TN={2}, FN={3}".format(tp, fp, tn, fn));
    print("Sensitivity: {0}".format(sensitivity));
    print("Specificity: {0}".format(specificity));
    print("Accuracy: {0}".format(accuracy));
    print("Balanced Accuracy: {0}".format(balanced_accuracy));
    print("Precision: {0}".format(precision));
    print("F1-Score: {0}".format(f1Score));
    print("MCC: {0}\n".format(MCC));

Print Class: 0
TP=393, FP=9607, TN=0, FN=0
Sensitivity: 1.0
Specificity: 0.0
Accuracy: 0.039
Balanced Accuracy: 0.5
Precision: 0.039
F1-Score: 0.076
MCC: nan

Print Class: 1
TP=0, FP=0, TN=9606, FN=394
Sensitivity: 0.0
Specificity: 1.0
Accuracy: 0.961
Balanced Accuracy: 0.5
Precision: nan
F1-Score: 0.0
MCC: nan

Print Class: 2
TP=0, FP=0, TN=9622, FN=378
Sensitivity: 0.0
Specificity: 1.0
Accuracy: 0.962
Balanced Accuracy: 0.5
Precision: nan
F1-Score: 0.0
MCC: nan

Print Class: 3
TP=0, FP=0, TN=9613, FN=387
Sensitivity: 0.0
Specificity: 1.0
Accuracy: 0.961
Balanced Accuracy: 0.5
Precision: nan
F1-Score: 0.0
MCC: nan

Print Class: 4
TP=0, FP=0, TN=9602, FN=398
Sensitivity: 0.0
Specificity: 1.0
Accuracy: 0.96
Balanced Accuracy: 0.5
Precision: nan
F1-Score: 0.0
MCC: nan

Print Class: 5
TP=0, FP=0, TN=9621, FN=379
Sensitivity: 0.0
Specificity: 1.0
Accuracy: 0.962
Balanced Accuracy: 0.5
Precision: nan
F1-Score: 0.0
MCC: nan

Print Class: 6
TP=0, FP=0, TN=9633, FN=367
Sensitivity: 0.0
Specifi

In [97]:
print("Overall Performance Prediction:");
print("Sensitivity: {0}%".format(round(mean(store_sens)*100, 4)));
print("Specificity: {0}%".format(round(mean(store_spec)*100, 4)));
print("Accuracy: {0}%".format(round(mean(store_acc)*100, 4)));
print("Balanced Accuracy: {0}%".format(round(mean(store_bal_acc)*100, 4)));
print("Precision: {0}%".format(round(mean(store_prec)*100, 4)));
print("F1-Score: {0}%".format(round(mean(store_fscore)*100, 4)))
print("MCC: {0}\n".format(round(mean(store_mcc), 4)))

Overall Performance Prediction:
Sensitivity: 3.8462%
Specificity: 96.1538%
Accuracy: 92.6038%
Balanced Accuracy: 50.0%
Precision: nan%
F1-Score: 0.2923%
MCC: nan



# The result of the Best Achieved accuracy seems very promisng, yet we go further to compare the results in details with WEKA results.

# MAE

In [98]:
import scipy
import numpy
from sklearn.metrics import mean_absolute_error

mean_absolute_error(y_actual_test_classes , y_pred_test_classes)
  





12.4811

In [100]:
import math
 
MSE = np.square(np.subtract(y_actual_test_classes, y_pred_test_classes)).mean() 
 
RMSE = math.sqrt(MSE)
RMSE

14.565956885834861

# The results from Convolutional Neural Network seems inconsistent! 

# While the accuracy seems extremely great, the mean absolute error and RMSE values are so bad that we can conclude it is not a good choice to treat the problem.